In [5]:
import requests
import json
import pandas as pd
import numpy as np
import pymongo

MONGO_URL = "mongodb://127.0.0.1:27017"
db = pymongo.MongoClient(MONGO_URL, connect=False, maxPoolSize=200).mindspore
access_token = "0c18bfc08a8aeece0c7ce2e37ab395cd"
def parseLinkHeader(headers):
    links = {}
    if "link" in headers:
        linkHeaders = headers["link"].split(", ")
        for linkHeader in linkHeaders:
            (url, rel) = linkHeader.split("; ")
            url = url[1:-1]
            rel = rel[5:-1]
            links[rel] = url
    return links

def getTotalPages(headers):
    return int(headers['total_page'])


def get_issues(owner, repo, state='open'):
    s = requests.session()
    s.keep_alive = False
    s.auth = ('', '')
    type = 'issues'
    request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/{type}?access_token={access_token}&state={state}&sort=created&direction=desc&page=1&per_page=20'
    response = s.get(request_url)
    pages = getTotalPages(response.headers)
    data = pd.DataFrame()
    for page in range(1, pages):
        request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/issues?access_token={access_token}&state={state}&sort=created&direction=desc&page={page}&per_page=20'
        response = s.get(request_url)
        contents = json.loads(response.text)
        for content in contents:
            data = data.append([{
                'id': content['id'],
                'number': content['number'],
                'state': content['state'],
                'userId': content['user']['id'],
                'userlogin': content['user']['login'],
                'username': content['user']['name'],
                'labels': content['labels'],
                'createTimestamp': content['created_at'],
                'updateTimestamp': content['updated_at'],
                'finishTimestamp': content['finished_at'],
                'type': content['issue_type']
            }], ignore_index=True)
        print(len(data))
    print(data.head(5))
    print('ok， now begin to write')
    data.to_csv(f"data/{owner}-{repo}-{type}.csv", encoding='utf8')

def get_issue_comments(owner, repos):
    s = requests.session()
    s.keep_alive = False
    s.auth = ('', '')
    request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/issues/comments?access_token={access_token}&sort=created&direction=asc&page=1&per_page=100'
    response = s.get(request_url)
    pages = getTotalPages(response.headers)
    data = pd.DataFrame()
    for page in range(1, pages):
        request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/issues/comments?access_token={access_token}&sort=created&direction=asc&page={page}&per_page=100'
        response = s.get(request_url)
        contents = json.loads(response.text)
        for content in contents:
            try:
                data = data.append([{
                    'issueNumber': content['target']['issue']['number'],
                    'userId': content['user']['id'],
                    'userlogin': content['user']['login'],
                    'username': content['user']['name'],
                    'createTimestamp': content['created_at'],
                    'updateTimestamp': content['updated_at'],
                }], ignore_index=True)
            except:
                print(content, page)
        print(len(data))
    print(data.head(5))
    print('ok， now begin to write')
    data.to_csv(f"data/issues_with_comments.csv", encoding='utf8')


def get_pulls(owner, repo, state='all'):
    s = requests.session()
    s.keep_alive = False
    s.auth = ('', '')
    type = 'pulls'
    request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/{type}?access_token={access_token}&state={state}&sort=created&direction=desc&page=1&per_page=100'
    response = s.get(request_url)
    print(response)
    pages = getTotalPages(response.headers)
    data = []
    for page in range(1, pages):
        request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/{type}?access_token={access_token}&state={state}&sort=created&direction=desc&page={page}&per_page=100'
        response = s.get(request_url)
        contents = json.loads(response.text)
        data.extend(contents)
        print(len(data))
    print('ok， now begin to write')
    db.all_pulls.insert_many(data)

def get_commits(owner, repo, state='open'):
    s = requests.session()
    s.keep_alive = False
    s.auth = ('', '')
    access_token = "a60a6091c65606487dfb7ae8033899a9"
    page = 1
    request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/commits?access_token={access_token}&page={page}&per_page=100'
    response = s.get(request_url)
    contents = json.loads(response.text)
    data = []
    while contents:   
        response = s.get(request_url)
        contents = json.loads(response.text)
        data.extend(contents)
        if len(data) % 500 == 0:
            print(len(data))
        page += 1
        request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/commits?access_token={access_token}&page={page}&per_page=100'
        response = s.get(request_url)
        contents = json.loads(response.text)
    print(data[0])
    print('ok， now begin to write')
    db.all_commits.insert_many(data)


if __name__ == '__main__':
    owner = 'mindspore'
    repo = 'mindspore'
    get_commits(owner, repo)
#     get_pulls(owner, repo)
    # get_issues(owner, repo)
    # get_contributors(owner, repo)
 

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


KeyboardInterrupt: 

In [14]:
def get_commits(owner, repo, state='open'):
    s = requests.session()
    s.keep_alive = False
    s.auth = ('', '')
    page = 1
    request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/commits?access_token={access_token}&page={page}&per_page=100'
    response = s.get(request_url)
    contents = json.loads(response.text)
    data = []
    while contents:   
        response = s.get(request_url)
        contents = json.loads(response.text)
        data.extend(contents)
        print(len(data))
        if len(data) % 500 == 0:
            print(len(data))
        page += 1
        request_url = f'https://gitee.com/api/v5/repos/{owner}/{repo}/commits?access_token={access_token}&page={page}&per_page=100'
        response = s.get(request_url)
        contents = json.loads(response.text)
    print(data[0])
    print('ok， now begin to write')
    db.commits.insert_many(data)

get_commits(owner, repo)

100
200
290
{'url': 'https://gitee.com/api/v5/repos/mindspore/mindspore/commits/2fd7f6710897e73b834d84e4ed68b9a216fb3351', 'sha': '2fd7f6710897e73b834d84e4ed68b9a216fb3351', 'html_url': 'https://gitee.com/mindspore/mindspore/commit/2fd7f6710897e73b834d84e4ed68b9a216fb3351', 'comments_url': 'https://gitee.com/api/v5/repos/mindspore/mindspore/commits/2fd7f6710897e73b834d84e4ed68b9a216fb3351/comments', 'commit': {'author': {'name': 'GinFung', 'date': '2020-03-30T09:54:05+08:00', 'email': 'yujianfeng5@huawei.com'}, 'committer': {'name': 'chang zherui', 'date': '2020-04-08T19:58:13+08:00', 'email': '760161589@qq.com'}, 'message': 'Add matmul biasadd fusion pass\n', 'tree': {'sha': 'eb5d139efbb52fa7ce9e54a6c2dc5becf0c37eae', 'url': 'https://gitee.com/api/v5/repos/mindspore/mindspore/git/trees/eb5d139efbb52fa7ce9e54a6c2dc5becf0c37eae'}}, 'author': {'id': 6574899, 'login': 'ginfung', 'name': 'YuJianfeng', 'avatar_url': 'https://gitee.com/assets/no_portrait.png', 'url': 'https://gitee.com/api/v